In [1]:
import pandas as pd 
import psycopg2
from spacy.matcher import PhraseMatcher
import spacy
from textblob import TextBlob
from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor
nlp = spacy.load('en_core_web_sm')

# get snippets

In [2]:
conn =psycopg2.connect(host="35.200.234.61",database="sales", user="postgres", password="cx6ac54nmgGtLD1y")

In [3]:
sql="select * from snippet where snippet.task_id in ( select task.id from task where task.actor in ( " \
      "select org_user.userid from org_user where org_user.organizationid = {})) order by snippet.id asc limit " \
      "10000".format(154)


In [4]:
snippet_df = pd.read_sql_query(sql,conn)

In [5]:
snippet_df.columns.to_list()

['id',
 'from_time',
 'to_time',
 'confidence',
 'text_',
 'speaker',
 'task_id',
 'language_code',
 'english_text',
 'transliterated_text',
 'is_final']

In [6]:
snippet_df.drop(columns=["confidence","language_code","english_text","transliterated_text","is_final"],inplace=True)

In [7]:
snippet_df.columns

Index(['id', 'from_time', 'to_time', 'text_', 'speaker', 'task_id'], dtype='object')

# Combine snippets based on speaker

In [33]:
def agent_customer_sequence(input_df):
    cached_snippets = []
    df = input_df
    if len(df) != 0:
        for i in range(len(df)):
            if len(cached_snippets) == 0:
                cached_snippets.append({"orignal_ids": [df["id"][i]], "speaker": df["speaker"][i], "text": df["text_"][i],
                                        "from_time": df["from_time"][i], "to_time": df["to_time"][i],
                                        "task_id": df["task_id"][i]})
            else:
                if df["speaker"][i] == cached_snippets[-1]['speaker'] and df["task_id"][i] == cached_snippets[-1]['task_id']:
                    cached_snippets[-1]["orignal_ids"].append(df["id"][i])
                    cached_snippets[-1]["text"] += ". " + df["text_"][i]
                    cached_snippets[-1]["to_time"] = df["to_time"][i]
                    cached_snippets[-1]["task_id"] = df["task_id"][i]
                else:
                    cached_snippets.append(
                        {"orignal_ids": [df["id"][i]], "speaker": df["speaker"][i], "text": df["text_"][i],
                         "from_time": df["from_time"][i], "to_time": df["to_time"][i], "task_id": df["task_id"][i]})
        new_snippets_df = pd.DataFrame(cached_snippets)
        return new_snippets_df
    else:
        return []    

In [34]:
new_df = agent_customer_sequence(snippet_df)

# make the question extraction methods

In [38]:
matcher = PhraseMatcher(nlp.vocab, attr='LOWER') #making a pattern matcher
question_terms = ["how","can","what","where","describe","who","when","why","?"]
patterns = [nlp(text) for text in question_terms]
matcher.add("question", None, *patterns)

In [39]:
def detect_sentences(text):
    if len(text.split()) > 0:
        text_blob = TextBlob(text)
        sentences=[]
        for sen in text_blob.sentences:
            sentences.append(str(sen))
        return sentences
    
    
def detect_questions(text):
    sentences = detect_sentences(text)
    question_parts=[]
    for sen in sentences:
        doc = nlp(sen)
        if len(matcher(doc)) !=0:
            matches = matcher(doc)
            if str(doc[matches[0][1]:matches[0][2]]) != "?":
                question_parts.append(str(doc[matches[0][1]:]))
            else:
                question_parts.append(str(sen))
    return question_parts
            
        

In [40]:
with ProcessPoolExecutor(max_workers=12) as exe:
    future = list(exe.map(detect_questions,new_df.text.to_list()))

In [41]:
new_df["question"]=future

In [43]:
def extract_pos(text):
    doc = nlp(text)
    nouns=[]
    verbs=[]
    for token in doc:
        if token.pos_ == "VERB":
            verbs.append(str(token.text))
        if token.pos_ == "NOUN":
            nouns.append(str(token.text))
    return nouns+verbs

In [51]:
def reporting(df):
    report={"snippet_ids":[],"speaker":[],"text":[],"type":[],"noun/verbs":[],"task_id":[]}
    old_taskid = df.task_id[0]
    for i in range(len(df)):
        if df.speaker[i] == "Agent" and len(df.question[i]) !=0:
            try:
                if df.task_id[i] == df.task_id[i+1]:
                    report['snippet_ids'].append(df.orignal_ids[i])
                    report['snippet_ids'].append(df.orignal_ids[i+1])
                    report['speaker'].append(df.speaker[i])
                    report['speaker'].append(df.speaker[i+1])
                    report['text'].append(df.text[i])
                    report['text'].append(df.text[i+1])
                    report['type'].append(df.question[i])
                    report['type'].append("response")
                    report['noun/verbs'].append([extract_pos(ques) for ques in df.question[i]])
                    report['noun/verbs'].append(extract_pos(df.text[i+1]))
                    report['task_id'].append(df.task_id[i])
                    report['task_id'].append(df.task_id[i+1])
                else:
                    report['snippet_ids'].append(df.orignal_ids[i])
                    report['snippet_ids'].append("")
                    report['speaker'].append(df.speaker[i])
                    report['speaker'].append("")
                    report['text'].append(df.text[i])
                    report['text'].append("")
                    report['type'].append(df.question[i])
                    report['type'].append("")
                    report['noun/verbs'].append([extract_pos(ques) for ques in df.question[i]])
                    report['noun/verbs'].append("")
                    report['task_id'].append(df.task_id[i])
                    report['task_id'].append("")
            except IndexError:
                pass 
    return report

                
                
            
            
            

In [52]:
report_df = reporting(new_df)

In [53]:
x=pd.DataFrame.from_dict(report_df)

In [54]:
x.to_excel("/home/andy/Desktop/noun_verb_list2.xlsx",index=None)

In [21]:
new_df.columns.to_list()

['orignal_ids',
 'speaker',
 'text',
 'from_time',
 'to_time',
 'task_id',
 'question']

In [23]:
for i in range(len(new_df.columns.to_list())):
    print(new_df.columns.to_list()[i+1])

speaker
text
from_time
to_time
task_id
question


IndexError: list index out of range